In [1]:
from repro_eval.Evaluator import RpdEvaluator, RplEvaluator
from repro_eval.util import arp, arp_scores
from repro_eval.util import print_base_adv, print_simple_line
from repro_eval.util import trim

import pytrec_eval
import numpy
import operator

import collections

In [2]:
qrels_file_path = '../../data/core18/qrels/qrels_common_core_2018_321_363.txt'
run_file_path = '../../data/core18/run_baseline_timo/run_timo_321_363'

In [3]:
# Import the original run
# The run is a nested dictionary with: key: topic ids, value: a dictionary
# The nested dictionary has: key: doc ids, value: doc scores
with open(run_file_path, 'r') as f_run:
    run_orig = pytrec_eval.parse_run(f_run)

# Import the qrels
# The qrels is a nested dictionary with: key: topic ids, value: a dictionary
# The nested dictionary has: key: doc ids, value: relevance labels
with open(qrels_file_path, 'r') as f_qrel:
    qrels = pytrec_eval.parse_qrel(f_qrel)

In [4]:
# Initialize the modified run
modified_run = collections.defaultdict(dict)

# Get the list of topic ids
topic_ids = numpy.array(list(run_orig.keys()))

print(topic_ids)

['321' '363']


In [5]:
# topics which need to be modified
for current_topic_id in topic_ids:

    # Get the list of document ids and scores for the given topic id
    # This is represented as a list of tuples (doc_id, score)
    current_ranking = list(run_orig[current_topic_id].items())
    
    # Sort the ranking as trec_eval
    # sort the tuples, score descending, doc_id descending
    # this is the same ordering used by trec_eval
    current_ranking.sort(key = operator.itemgetter(0), reverse = True)         # doc_id descending
    current_ranking.sort(key = operator.itemgetter(1), reverse = True)         # score descending

    # Create a copy of the current ranking
    # The score is copied as a string
    modified_ranking = numpy.array(current_ranking)
    
    # Add the modified ranking to the dictionary
    # for each element of the new ranking
    for row in modified_ranking:
        # get the current document id
        current_doc_id = row[0]
        # get the current document score
        current_score = row[1]
        # add the element (doc_id, score) to the run dictionary
        # the score has to be converted to float
        modified_run[current_topic_id][current_doc_id] = float(current_score)

In [6]:
print(modified_run)

defaultdict(<class 'dict'>, {'321': {'e9aea1113e11f4ee20694f42e8505d85': 21.8298, '81a3cf19017aca2cc50bcff202719ffe': 21.796301, '34c01b3aa97d1257ced5bb4fe7fda816': 21.734699, '46645b7b3bc0b5286ee283f1fb3e0532': 21.734697, '44e6fb86af809b7d01e68fa68f3aa231': 21.734697, 'a1d50a2ee98c7327b9fe4af429687acd': 21.734695, '5845ed2da07a29ad086f2d116113c320': 20.6903, 'aaa6d7ebbdb2318e4339ad47e2e00ff6': 20.690298, 'eacd327b20aa77a2aa909596ae336497': 20.3053, '3ee24c80-7d84-11e1-a01f-52d5c6187537': 20.0257, '705b0b6c9f00adb467be285e36dca522': 20.020901, 'ebf573d0-50dd-11e1-bd4f-8a7d53f6d6c2': 19.8263, '70c1790a-45d6-11e1-8cb4-cd666424bf3a': 19.590599, '550907ec43d4712f37f18867afb01739': 19.515699, 'cc01db418210d9797784adfb3f14bd39': 19.2787, 'a6f35002-8ad2-11e1-b132-eab19f4e8b2e': 19.1182, 'e2750f602aca39c08ce6b4220ade59aa': 19.026199, '7468dad4-76a1-11e1-862c-76c4e0e725cd': 18.869301, '8e123fde1ffaaf2184f01e00fee60810': 18.8561, 'aab75b44-b971-11e5-85cd-5ad59bc19432': 18.841801}, '363': {'2686c

In [7]:
print(run_orig)

defaultdict(<class 'dict'>, {'321': {'e9aea1113e11f4ee20694f42e8505d85': 21.8298, '81a3cf19017aca2cc50bcff202719ffe': 21.796301, '34c01b3aa97d1257ced5bb4fe7fda816': 21.734699, '44e6fb86af809b7d01e68fa68f3aa231': 21.734697, '46645b7b3bc0b5286ee283f1fb3e0532': 21.734697, 'a1d50a2ee98c7327b9fe4af429687acd': 21.734695, '5845ed2da07a29ad086f2d116113c320': 20.6903, 'aaa6d7ebbdb2318e4339ad47e2e00ff6': 20.690298, 'eacd327b20aa77a2aa909596ae336497': 20.3053, '3ee24c80-7d84-11e1-a01f-52d5c6187537': 20.0257, '705b0b6c9f00adb467be285e36dca522': 20.020901, 'ebf573d0-50dd-11e1-bd4f-8a7d53f6d6c2': 19.8263, '70c1790a-45d6-11e1-8cb4-cd666424bf3a': 19.590599, '550907ec43d4712f37f18867afb01739': 19.515699, 'cc01db418210d9797784adfb3f14bd39': 19.2787, 'a6f35002-8ad2-11e1-b132-eab19f4e8b2e': 19.1182, 'e2750f602aca39c08ce6b4220ade59aa': 19.026199, '7468dad4-76a1-11e1-862c-76c4e0e725cd': 18.869301, '8e123fde1ffaaf2184f01e00fee60810': 18.8561, 'aab75b44-b971-11e5-85cd-5ad59bc19432': 18.841801}, '363': {'2686c

In [8]:
print(modified_run == run_orig)

True


In [9]:
# Initialize the Rpd evaluator object
rpd_eval = RpdEvaluator()
# Set all the field of the rpd object
rpd_eval.run_b_orig = run_orig
rpd_eval.run_b_rep = modified_run

rpd_eval.rel_eval = pytrec_eval.RelevanceEvaluator(qrels, pytrec_eval.supported_measures)

rpd_eval.trim()
rpd_eval.evaluate()

In [10]:
# Compute Kndall's tau
ktu = rpd_eval.ktau_union()

In [11]:
print(ktu)

{'baseline': {'321': 0.98947368421053, '363': 1.0}}


In [ ]:
# Import from files
rpd_eval = RpdEvaluator(qrel_orig_path=qrels_file_path,
                        run_b_orig_path=run_file_path,
                        run_b_rep_path=run_file_path)

In [ ]:
rpd_eval.trim()
rpd_eval.evaluate()

# KTU
ktau = rpd_eval.ktau_union()
print("Kendall's tau Union (KTU)")
print('------------------------------------------------------------------')
for topic, value in ktau.get('baseline').items():
    print_base_adv(topic, 'KTU', value, ktau.get('baseline').get(topic))
print_base_adv('ARP', 'KTU', arp(ktau.get('baseline')), arp(ktau.get('baseline')))